In [ ]:
try:
    import google.colab  # noqa: F401
except ImportError:
    import dolfin
    import mshr
else:
    try:
        import dolfin
        import mshr
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
        import dolfin
        import mshr

In [ ]:
domain = mshr.Rectangle(dolfin.Point(0., 0.), dolfin.Point(1., 1.))
mesh = mshr.generate_mesh(domain, 32)

In [ ]:
subdomains = dolfin.MeshFunction("size_t", mesh, 2)
subdomains.set_all(0)

In [ ]:
class Left(dolfin.SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and abs(x[0]) < dolfin.DOLFIN_EPS


class Right(dolfin.SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and abs(x[0] - 1.) < dolfin.DOLFIN_EPS


class Bottom(dolfin.SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and abs(x[1]) < dolfin.DOLFIN_EPS


class Top(dolfin.SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and abs(x[1] - 1.) < dolfin.DOLFIN_EPS

In [ ]:
boundaries = dolfin.MeshFunction("size_t", mesh, mesh.topology().dim() - 1)
boundaries.set_all(0)
bottom = Bottom()
bottom.mark(boundaries, 1)
left = Left()
left.mark(boundaries, 2)
top = Top()
top.mark(boundaries, 3)
right = Right()
right.mark(boundaries, 4)

In [ ]:
import numpy as np

In [ ]:
unique, counts = np.unique(boundaries.array(), return_counts=True)
d = dict(zip(unique, counts))
for label in range(5):
    assert d[label] > 0